## 测试文件-测试本电脑能否和远程VLLM框架部署的服务器进行连接

### 配置ssh密钥
两台电脑构建ssh链接-远程服务器的配置文件
    HostName 'IP地址'
    User '用户名'
    Port '端口号'
    IdentityFile '识别文件的路径'
    IdentitiesOnly yes'是否显示指定的密钥'
    ServerAliveInterval 60 '服务器连接超时的时间'
    ServerAliveCountMax 3 '服务器连接超时重试次数

### SSH连接本端网内的服务器
ssh -N -L 18000:127.0.0.1:8000 mylinux

这里18000为本端端口，8000为服务器端口，mylinux为ssh配置文件中设置的服务器用户名(~/.ssh/config里配置的服务器别名)

可在本机电脑使用powershell使用ssh -G admin-204 | Select-Object -First 20 判断本地电脑配置的服务器信息是否正确

后续输入：ssh -N -L 18000:127.0.0.1:8000 mylinux 创建ssh通道链接到本地的服务器

In [ ]:
import httpx
from openai import OpenAI

client = OpenAI(
    api_key="你自己设定的api_key", 
    base_url="http://127.0.0.1:18000/v1", # api地址
    http_client=httpx.Client(trust_env=False, timeout=120.0), # 忽略本地环境的代理
)

resp = client.chat.completions.create(
    model="Qwen3-8B",# 本地部署的大模型
    messages=[{"role":"user","content":"请用一句话介绍你自己。"}],
    max_tokens=256,
    temperature=0.5,
)

print(resp.choices[0].message.content)


<think>
嗯，用户让我用一句话介绍自己。首先，我需要确定用户的需求是什么。他们可能是在测试我的能力，或者想快速了解我的功能。作为通义千问，我应该突出我的核心特点，比如多语言支持、广泛的知识库和强大的推理能力。同时，要保持简洁，避免信息过载。还要考虑用户可能的深层需求，比如他们是否在寻找一个可靠的助手，或者需要处理复杂任务。因此，我需要确保这句话既全面又精炼，涵盖关键点，比如语言能力、知识范围、应用场景和交互方式。还要注意用词准确，比如“多语言支持”而不是“多种语言”，更专业。最后检查是否符合要求，确保没有多余信息，保持自然流畅。
</think>

我是通义千问，一个由通义实验室开发的超大规模语言模型，能够支持多种语言，具备广泛的知识和强大的推理能力，可以协助用户完成各种任务和解答问题。


In [ ]:
### 使用langchain_openai库调用本地部署的大模型
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "你自己设定的api_key" # api地址
os.environ["OPENAI_BASE_URL"] = "http://127.0.0.1:18000/v1"
# 推荐：确保 localhost 不走代理（避免你之前的 502）
os.environ["NO_PROXY"] = "127.0.0.1,localhost"

llm = ChatOpenAI(
    model="Qwen3-8B",
    temperature=0.5,
    max_tokens=256,
)

print(llm.invoke("请用一句话介绍你自己。").content)


<think>
嗯，用户让我用一句话介绍自己。首先，我需要确定用户的需求是什么。他们可能是在测试我的能力，或者想快速了解我的功能。作为通义千问，我应该突出我的核心特点，比如多语言支持、广泛的知识库和强大的推理能力。同时，要简洁明了，避免冗长。可能用户希望这句话能涵盖我的主要功能，比如回答问题、创作内容、逻辑推理等。还要注意用词准确，比如“通义千问”是我的名字，需要提到。另外，可能需要强调我的应用场景，比如日常交流、学习、工作等。需要确保句子结构流畅，信息全面但不过于复杂。有没有遗漏的关键点？比如是否要提到我的训练数据或技术背景？不过用户可能只需要功能性的介绍，不需要技术细节。所以综合起来，应该包括名称、功能、应用场景和优势。现在组织成一句话，确保自然流畅。
</think>

我是通义千问，一个由通义实验室开发的超大规模语言模型，能够支持多语言交互，具备广泛的知识储备和强大的逻辑推理能力，可协助用户完成文本创作、问题解答、编程指导等多种任务。
